<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Get activities from contact
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Get+activities+from+contact:+Error+short+description">Bug report</a>

**Tags:** #hubspot #contact #activity #notes #emails #communications #meetings #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-21 (Created: 2023-08-21)

**Description:** This notebook demonstrates how to retrieve all activities from a contact URL by combining various HubSpot API endpoints.

**References:**
- [HubSpot API - Contacts](https://developers.hubspot.com/docs/api/crm/contacts)
- [HubSpot API - Associations v4](https://developers.hubspot.com/docs/api/crm/associations)
- [HubSpot API - Communications](https://developers.hubspot.com/docs/api/crm/communications)

## Input

### Import libraries

In [ ]:
import requests
import naas
from naas_drivers import hubspot
import pandas as pd

### Setup variables
**Mandatory**

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `contact_url`: This variable stores the HubSpot contact URL.

In [ ]:
# Mandatory
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
contact_url = "https://app.hubspot.com/contacts/2474088/record/0-1/666001" # "https://app.hubspot.com/contacts/xxxxx/record/0-1/xxxxx"

## Model

### Get contact ID from URL

In [ ]:
def get_contact_ID_from_URL(url):
    # Init
    uid = url
    
    # Check if URL is valid
    if not url.startswith("https://app.hubspot.com/contacts/"):
        raise BaseException("HubSpot URL Invalid! It must start by https://app.hubspot.com/contacts/")
    
    # Split URL to get ID
    if "/record/0-1/" in url:
        uid = url.split("/record/0-1/")[-1].split("/")[0]
    return uid

contact_id = get_contact_ID_from_URL(contact_url)
print("Contact ID:", contact_id)

### Functions to get contact associations details

In [ ]:
def get_association_from_contact(
    token,
    contact_id,
    endpoint,
):
    # Init
    results = []
    
    # Requests
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v4/objects/contacts/{contact_id}/associations/{endpoint}"
    
    # Response
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        results = res.json().get("results")
    return results

def retrieve_object_details(
    token,
    object_id,
    object_type,
    properties=None,
):
    # Init
    data = []
    params = {
        "archived": "false"
    }
    
    # Requests
    if properties:
        params["properties"] = properties
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v3/objects/{object_type}/{object_id}"
    
    # Response
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        data = res.json().get("properties")
    else:
        print(res.text)
    return pd.DataFrame([data])

def create_activity_df(
    token,
    contact_id,
    activity,
    properties_dict=None,
):
    # Init
    df = pd.DataFrame()
    if not properties_dict:
        properties_dict = {
            "hs_object_id": "activity_hs_id",
            "hs_lastmodifieddate": "activity_date",
            "hs_body_preview": "activity_content"
        }
    properties = [x for x in properties_dict]
    
    # List activities
    data = get_association_from_contact(
        token,
        contact_id,
        activity
    )
    print("Data fetched:", len(data))
    for d in data:
        object_id = d.get("toObjectId")
        tmp_df = retrieve_object_details(
            token,
            object_id,
            activity,
            properties
        )
        if len(tmp_df) > 0:
            tmp_df = tmp_df[properties]
        df = pd.concat([df, tmp_df])
    if len(df) > 0:
        df = df.rename(columns=properties_dict)
        if 'activity_type' not in df:
            df.insert(loc=1, column="activity_type", value=activity.upper())
    return df.reset_index(drop=True)

### Get notes associated to contact

In [ ]:
df_notes = create_activity_df(
    hs_access_token,
    contact_id,
    "notes",
)
df_notes

### Get sales emails associated to contact

In [ ]:
df_sales_emails = create_activity_df(
    hs_access_token,
    contact_id,
    "emails",
)
df_sales_emails

### Get meetings associated to contact

In [ ]:
properties_dict = {
    "hs_object_id": "activity_hs_id",
    "hs_lastmodifieddate": "activity_date",
    "hs_meeting_title": "activity_content"
}

df_meetings = create_activity_df(
    hs_access_token,
    contact_id,
    "meetings",
    properties_dict,
)
df_meetings

### Get communications associated to contact

In [ ]:
properties_dict = {
    "hs_object_id": "activity_hs_id",
    "hs_lastmodifieddate": "activity_date",
    "hs_communication_channel_type": "activity_type",
    "hs_body_preview": "activity_content"
}

df_communications = create_activity_df(
    hs_access_token,
    contact_id,
    "communications",
    properties_dict
)
df_communications

### Create activity DataFrame with contact details

In [ ]:
def get_contact_details(
    hs_access_token,
    contact_id,
    properties
):
    # Init
    data = {}
    
    # Get contact
    contact = hubspot.connect(hs_access_token).contacts.get(contact_id, hs_properties=properties)
    
    # Return properties
    if len(contact) > 0:
        data = contact.get("properties")
    return data

def create_activity_df(
    contact_id,
    df_notes,
    df_sales_emails,
    df_meetings,
    df_communications,
):
    # Init
    df = pd.DataFrame()
    
    # Concat
    df = pd.concat(
        [
            df_notes,
            df_sales_emails,
            df_meetings,
            df_communications
        ]
    )
    
    # Format date
    df["activity_date"] = pd.to_datetime(df["activity_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    df = df.sort_values(by="activity_date").reset_index(drop=True)
    
    # Add contact details
    contact_properties = ["hs_object_id", "firstname", "lastname", 'email', 'linkedinbio', 'jobtitle']
    contact = get_contact_details(
        hs_access_token,
        contact_id,
        contact_properties
    )
    for i, x in enumerate(contact_properties):
        df.insert(loc=i, column=x, value=contact.get(x))
    pd.set_option('display.max_colwidth', None)
    return df

df_activity = create_activity_df(
    contact_id,
    df_notes,
    df_sales_emails,
    df_meetings,
    df_communications,
)
print("Activities fetched:", len(df_activity))
df_activity.tail(1)

## Output

### Save DataFrame to CSV

In [ ]:
df_activity.to_csv(f"contact_activity_{contact_id}.csv", index=False)